In [1]:
import pandas as pd
import numpy as np
tmp = pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")
tmp

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563
...,...,...,...
5154466,138493,44022,1260209449
5154467,138493,4958,1260209482
5154468,138493,68319,1260209720
5154469,138493,40819,1260209726


In [2]:
users = list(set(tmp.loc[:,'user']))
users.sort()
users_dict = {users[i]: i for i in range(len(users))}
tmp['user']  = tmp['user'].map(lambda x : users_dict[x])
tmp

,user,item,time
0,0,4643,1230782529
1,0,170,1230782534
2,0,531,1230782539
3,0,616,1230782542
4,0,2140,1230782563
...,...,...,...
5154466,31359,44022,1260209449
5154467,31359,4958,1260209482
5154468,31359,68319,1260209720
5154469,31359,40819,1260209726


In [3]:

items =  list(set((tmp.loc[:, 'item'])))
items.sort()

items_dict = {items[i]: i for i in range(len(items))}
tmp['item']  = tmp['item'].map(lambda x : items_dict[x])

In [4]:
tmp = tmp.groupby("user")["item"]
tmp

In [5]:
first = True
for u, items in tmp:
    a = pd.DataFrame({"user":[u]*len(items), "item":items})
    if first:
        first =False
        txt = a
    else:
        txt = pd.concat([txt,a], axis = 0, sort = False)

In [6]:
before = None
grouped_txt = txt.groupby("user")
train_prop = 0.8
with open('./train.txt','a') as tr, open('./test.txt', 'a') as te:
    for u, group in grouped_txt:
        if before!=None:
            tr.write("\n")
            te.write("\n")
        tr.write(str(u))
        te.write(str(u))
        before = u
        n_items_u = len(group)
        idx = np.zeros(n_items_u, dtype='bool')
        idx[np.random.choice(n_items_u, size=int(train_prop * n_items_u), replace=False).astype('int64')] = True
        for train_item in group[idx]["item"]:
            tr.write(" "+str(train_item))
        for test_item in group[np.logical_not(idx)]["item"]:
            te.write(" "+str(test_item))



In [7]:
before = None
grouped_txt = txt.groupby("user")
train_prop = 0.8
with open('./submission.txt','a') as sb:
    for u, group in grouped_txt:
        if before!=None:
            sb.write("\n")
        sb.write(str(u))
        before = u
        n_items_u = len(group)
        idx = np.zeros(n_items_u, dtype='bool')
        idx[np.random.choice(n_items_u, size=int(train_prop * n_items_u), replace=False).astype('int64')] = True
        for train_item in group[idx]["item"]:
            tr.write(" "+str(train_item))
        for test_item in group[np.logical_not(idx)]["item"]:
            te.write(" "+str(test_item))


ValueError: I/O operation on closed file.

In [6]:
before = None
with open('./submission.txt','a') as sb:
    for u,i in zip(txt["user"], txt["item"]):
        if before != u:
            if before!=None:
                sb.write("\n")
            sb.write(str(u))
            before = u
        sb.write(" "+str(i))
